In [56]:
import sys
import codecs

### Challege 1
#### Convert Hex to Base64
Test string: 49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d

Ouput: SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t

In [57]:
def HexToB64(bytes_Hex):
    hex_Decoded = codecs.decode(bytes_Hex, 'hex')
    b64_Encoded = codecs.encode(hex_Decoded, 'base64')
    return b64_Encoded

In [58]:
bstr_test = b'49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
print(HexToB64(bstr_test))

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t\n'


### Challege 2
#### Fixed XOR
Write a function that takes two equal-length buffers and produces their XOR combination.

If your function works properly, then when you feed it the string:

1c0111001f010100061a024b53535009181c
... after hex decoding, and when XOR'd against:

686974207468652062756c6c277320657965
... should produce:

746865206b696420646f6e277420706c6179


In [59]:
def FixedXOR(byte_str1, byte_str2):
    hex_str1 = int(str(byte_str1.decode('utf-8')), base=16)
    hex_str2 = int(str(byte_str2.decode('utf-8')), base=16)
    val_XOR = hex(hex_str1 ^ hex_str2).encode()[2:]
    return val_XOR

In [60]:
byte_teststr1 = b'1c0111001f010100061a024b53535009181c'
byte_teststr2 = b'686974207468652062756c6c277320657965'
print(FixedXOR(byte_teststr1, byte_teststr2))

b'746865206b696420646f6e277420706c6179'


### Challege 3
#### Single-byte XOR Cypher

The hex encoded string:

1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736
... has been XOR'd against a single character. Find the key, decrypt the message.

You can do this by hand. But don't: write code to do it for you.

In [64]:
def GetStringLetterFreq(str_Test, dict_EnglishLetterFreq):
    """
    Input: string of letters, a dictionary containing the letters to be tested as keys
    Output: a dictionary of the frequency of letters in the string
    Notes: Called by EnglishScore
    """
    list_EnglishLetters = [i for i, k in dict_EnglishLetterFreq.items()]
    dict_StringLetterCount = dict.fromkeys(list_EnglishLetters , 0)
    str_Test = str_Test.lower()
    val_NotLetterPenalty = 0
    for c in str_Test:
        if c in list_EnglishLetters:
            dict_StringLetterCount[c] += 1
        elif c == ' ':
            val_NotLetterPenalty += 0 #no penalty for spaces
        else:
            val_NotLetterPenalty += 0.1 #adjustable penalty for non-English letter, non-space characters
    dict_StringLetterFreq = {k: v / len(str_Test) for k, v in dict_StringLetterCount.items()}
    return dict_StringLetterFreq, val_NotLetterPenalty

def EnglishScore(str_Test):
    dict_EnglishLetterFreq = {'a': 0.0817,'b': 0.0149,'c': 0.0278,'d': 0.0425,'e': 0.127,'f': 0.0223,'g': 0.0202,
                              'h': 0.0609,'i': 0.0697,'j': 0.0015,'k': 0.0077,'l': 0.0403,'m': 0.0241,'n': 0.0675,
                              'o': 0.0751,'p': 0.0193,'q': 0.001,'r': 0.0599,'s': 0.0633,'t': 0.0906,'u': 0.0276,
                              'v': 0.0098,'w': 0.0236,'x': 0.0015, 'y': 0.0197,'z': 0.0007}
    dict_TestLetterFreq, val_NotLetterPenalty = GetStringLetterFreq(str_Test, dict_EnglishLetterFreq)
    list_EnglishScore = [(a - b)**2 for (a,b) in zip(dict_TestLetterFreq.values(), dict_EnglishLetterFreq.values())]
    total_EnglishScore = sum(list_EnglishScore) + val_NotLetterPenalty
    return total_EnglishScore

def ScoreStringList(list_TestStrings):
    dict_StringScores = dict.fromkeys(list_TestStrings, 0)
    for s in list_TestStrings:
        dict_StringScores[s] = EnglishScore(s)
    string_MinScore = min(dict_StringScores, key=lambda key: dict_StringScores[key])
    return string_MinScore, dict_StringScores[string_MinScore], dict_StringScores

In [100]:
def SingleByteXOR(str_Encoded, byte_Char):
    hex_SingleChar = int(byte_Char.decode(), base=16)
    list_Decoded = []
    for c in codecs.decode(str_Encoded, 'hex'):
        val_XOR = hex(c ^ hex_SingleChar)[2:]
        list_Decoded.append(val_XOR)
    str_Decoded = ''.join(list_Decoded)
    try:
        str_UniDecoded = codecs.decode(str_Decoded, 'hex').decode('ascii')
    except:
        str_UniDecoded = ''
    return str_UniDecoded


def DecodeSingleByteXOR(str_HexEncode):
    list_DecodedStrings = []
    for i in range(0, 256):
        test_Byte = codecs.encode(chr(i).encode(), 'hex')
        bytes_XORd = SingleByteXOR(str_HexEncode, test_Byte)
        if len(bytes_XORd) > 0:
            list_DecodedStrings.append(bytes_XORd)
    return list_DecodedStrings

In [101]:
str_SingleByteEncode = b'1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
list_DecodedStrings = DecodeSingleByteXOR(str_SingleByteEncode)
str_BestEngScore, val_BestEngScore, dict_AllScores = ScoreStringList(list_DecodedStrings)
print(str_BestEngScore)

Cooking MC's like a pound of bacon


### Challenge 4
#### Detect single-character XOR
One of the 60-character strings in this file has been encrypted by single-character XOR.

In [114]:
f = open('./ChallengeSet1_4.txt', 'r')
list_EncodedStrings = [l.strip() for l in f]

In [115]:
list_EncodedStrings

['...']

In [116]:
list_AllDecodedStrings = []
for s in list_EncodedStrings:
    list_DecodedElement = DecodeSingleByteXOR(s)
    if len(list_DecodedElement) > 0:
        str_BestEngScore, val_BestEngScore, dict_AllScores = ScoreStringList(list_DecodedElement)
        print(str_BestEngScore)
        list_AllDecodedStrings.extend(list_DecodedElement)


ui|;ynr2:rFyQ0QvH&ZXbu)7SeYqP
B;+PYl!CFN<T&f VK+OzQHKgYn|/
Ri$o+C5uh| [JfwZ%l #'#cTyn&u
sRJIU\IIUXM\OIDTNWHPMTSZ7
Ea NEy2HcAoF2UmCUxe%s)Sv69KQL
Q7]Kl(X4wQL\gU*;7ej  XQ0OjkhtD
wpUs{gnns F8b-aT2*zRAh{Gj>xuED
iUhkZHXPIKtXKAnQWsAD>XtbxIK
[$LrLi;hsDbv!5 j$OaEfG=>rKDl=S


In [105]:
str_BestEngScore, val_BestEngScore, dict_AllScores = ScoreStringList(list_AllDecodedStrings)
print(str_BestEngScore)

iUhkZHXPIKtXKAnQWsAD>XtbxIK


In [106]:
[(k, dict_AllScores[k]) for k in sorted(dict_AllScores, key = dict_AllScores.get)]

[('iU\x1chkZHXPIK\x16tXKAnQWs\x1bAD>XtbxIK', 0.46956448222222225),
 ('Iu<HKzhxpik6TxkaNqwS;ad\x1exTBXik', 0.46956448222222225),
 ('kW\x1ejiXJZRKI\x14vZIClSUq\x19CF<Zv`zKI', 0.5918067044444444),
 ('Kw>JIxjzrki4VzicLsuQ9cf\x1czV@Zki', 0.5918067044444444),
 ('Ea NEy2HcAoF2Um\x7fCUxe%s)Sv69KQL', 0.7314089266666666),
 ('wpUs{gnns F8b-aT2*zRAh{Gj>xuED', 0.7353911488888889),
 ('tsVpxdmmp#E;a.bW1)yQBkxDi={vFG', 0.7543444822222222),
 ('urWqyellq"D:`/cV0(xPCjyEh<zwGF', 0.7547444822222222),
 ('hT\x1dij[IYQHJ\x17uYJ@oPVr\x1a@E?YucyHJ', 0.77068226),
 ('Ht=IJ{iyqhj7Uyj`OpvR:`e\x1fyUCYhj', 0.77068226),
 ('dX\x11efWEU]DF\x1byUFLc\\Z~\x16LI3UyouDF', 0.7711111488888889),
 ('Dx1EFweu}df;YuflC|z^6li\x13uYOUdf', 0.7711111488888889),
 ('ebGaiu||a2T*p?sF 8h@SziUx,jgWV', 0.8401600377777777),
 ('afCemqxxe6P.t;wB$<lDW~mQ|(ncSR', 0.8424800377777777),
 ('sRJ\x1dIU\\I\x1dIUX\x1dM\\OID\x1dTN\x1dWHPMTSZ7', 0.8426889266666666),
 ('Srj=iu|i=iux=m|oid=tn=whpmtsz\x17', 0.8426889266666666),
 ('uTL\x1bOSZO\x1bOS^\x1bKZIOB

In [37]:
len(dict_AllScores)

256